In [1]:
import pickle

In [2]:
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [5]:
data = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
val = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [6]:
len(data)

1154112

In [7]:
data

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


In [8]:
data['duration'] = data.dropOff_datetime - data.pickup_datetime
data.duration = data.duration.apply(lambda td:td.total_seconds()/60)

In [9]:
val['duration'] = val.dropOff_datetime - val.pickup_datetime
val.duration = val.duration.apply(lambda td:td.total_seconds()/60)

In [10]:
data.duration.mean()

19.1672240937939

In [11]:
data_v1 = data[(data.duration >= 1) & (data.duration <= 60)]

In [12]:
val_v1 = val[(val.duration >= 1) & (val.duration <= 60)]

In [13]:
data_v2  = data_v1[['PUlocationID','DOlocationID']]

In [14]:
val_v2  = val_v1[['PUlocationID','DOlocationID']]

In [15]:
len(data) - len(data_v2)

44286

In [16]:
100 * data_v2.isna().sum()/len(data_v2)

PUlocationID    83.527328
DOlocationID    13.327044
dtype: float64

In [17]:
data_v2.fillna(-1,inplace=True)

C:\OLAMIDE\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [18]:
val_v2.fillna(-1,inplace=True)

In [19]:
data_v2  = data_v2[['PUlocationID','DOlocationID']].astype(str)

In [20]:
val_v2  = val_v2[['PUlocationID','DOlocationID']].astype(str)

In [21]:
dv = DictVectorizer()

In [22]:
train_dicts = data_v2 .to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [23]:
y_dicts = val_v2 .to_dict(orient='records')
y_test = dv.transform(y_dicts)

In [24]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [25]:
len(dv.feature_names_)

525

In [26]:
target = 'duration'
y_train = data_v1[target].values

In [27]:
target = 'duration'
y_val = val_v1[target].values

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107211832

In [29]:
val_pred = lr.predict(y_test)
mean_squared_error(y_val, val_pred, squared=False)

11.014283203471416